In [1]:
import os
import sys
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)

import pandas as pd
import numpy as np
from sklearn.datasets import load_files

from datacode.retrieve_data import pull_data
from nltk.corpus import stopwords
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
import spacy
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim

from features.pre_process import set_stop_words, sent_to_words, remove_stopwords, bigrams, get_corpus
import pickle
import warnings
from models.text_model import gen_lda_model, train_vectors



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/topper/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/topper/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
root_dir = os.path.dirname(os.path.dirname(os.path.abspath("LICENSE")))
interim_data_path = os.path.join(root_dir, "data/interim")
processed_data_path = os.path.join(root_dir, "data/processed")
raw_data_path = os.path.join(root_dir, "data/raw")

In [3]:
# Pull in training data, split off features and label data and save the features array as a df
imdb_train = pull_data(os.path.join(raw_data_path, "aclImdb/train"))
text_train, y_train = imdb_train.data, imdb_train.target
text_train_df = pd.DataFrame({'text': text_train})

# SAMPLING STEP - FOR TEST

In [4]:
text_train_df_samp = text_train_df.head(1000)

In [5]:
# Better to add to a YAML or other text file in due course
added_stop_words = ['film','movie','picture','review']

In [6]:
# Bringing in a pipeline pre-trained on written web text, includes vocab vectors, syntax and entities. 
#nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

In [7]:
# Run get_corpus which 
#train_corpus4, train_id2word4, bigram_train4 = get_corpus(text_train_df, added_stop_words)
train_corpus4, train_id2word4, bigram_train4 = get_corpus(text_train_df_samp, added_stop_words)  # REVERT AFTER TEST

In [8]:
corpus_path = os.path.join(interim_data_path, "train_corpus4.pkl")
id2word_path = os.path.join(interim_data_path, "train_id2word4.pkl")
bigram_train_path = os.path.join(interim_data_path, "bigram_train4.pkl")

In [9]:
# Keep the files
with open(corpus_path, 'wb') as f:
    pickle.dump(train_corpus4, f)
with open(id2word_path, 'wb') as f:
    pickle.dump(train_id2word4, f)
with open(bigram_train_path, 'wb') as f:
    pickle.dump(bigram_train4, f)

In [10]:
# Bring in previously stored files 
with open(corpus_path, 'rb') as f:
    train_corpus4 = pickle.load(f)
with open(id2word_path, 'rb') as f:
    train_id2word4 = pickle.load(f)
with open(bigram_train_path, 'rb') as f:
    bigram_train4 = pickle.load(f)

In [11]:
model_log = os.path.join(processed_data_path, "logs/lda_model.log")
model_data = os.path.join(processed_data_path, "model_data/lda_train4.model")

lda_train4 = gen_lda_model(train_corpus4, train_id2word4, model_log, model_data)

In [12]:
train_vecs = train_vectors(lda_train4, train_corpus4, text_train_df_samp)